In [1]:
from transformers import TimesformerForVideoClassification

In [354]:
model = TimesformerForVideoClassification.from_pretrained("facebook/timesformer-base-finetuned-k400")

In [355]:
import torch
import os
import cv2
import random
import numpy as np

In [356]:
IMG_SIZE = 16

In [357]:
def frames_from_video_file(video_path, MAX_SEQ_LENGTH, frame_step, output_size = (IMG_SIZE, IMG_SIZE)):
    result = []
  
    src = cv2.VideoCapture(str(video_path))  

    video_length = src.get(cv2.CAP_PROP_FRAME_COUNT)

    need_length = 1 + (MAX_SEQ_LENGTH - 1) * frame_step

    if need_length > video_length:
        start = 0
    else:
        max_start = video_length - need_length
        start = random.randint(0, max_start + 1)

    src.set(cv2.CAP_PROP_POS_FRAMES, start)
    # ret is a boolean indicating whether read was successful, frame is the image itself
    ret, frame = src.read()
    frame = cv2.resize(frame, output_size)
    cv2.normalize(frame, frame, 0, 255, cv2.NORM_MINMAX)
    frame = frame.transpose(2,1,0)
    frame = torch.tensor(frame)
    frame = frame.to(torch.float32)
    result.append(frame)

    for _ in range(MAX_SEQ_LENGTH - 1):
        for _ in range(frame_step):
            ret, frame = src.read()
        if ret:
            frame = cv2.resize(frame, output_size)
            cv2.normalize(frame, frame, 0, 255, cv2.NORM_MINMAX)
            frame = frame.transpose(2,1,0)
            frame = torch.tensor(frame)
            frame = frame.to(torch.float32)
            result.append(frame)
        else:
            frame = np.zeros_like(result[0])
            frame = torch.tensor(frame)
            frame = frame.to(torch.float32)
            result.append(frame)
    src.release()
    #result = np.array(result)[..., [2, 1, 0]]

    return result

In [358]:
class VideoDataset(torch.utils.data.Dataset):
    def __init__(self, df, root_dir, MAX_SEQ_LENGTH, frame_step):
        self.video_paths = df["video-name"].values.tolist()
        self.labels = df["label"].values.tolist()
        self.n_frames = MAX_SEQ_LENGTH
        self.root_dir = root_dir
        self.frame_step = frame_step
        
    def __len__(self):
        return len(self.video_paths)
    
    def __getitem__(self, idx):
        path = self.video_paths[idx]
        label = self.labels[idx]
        frames = frames_from_video_file(os.path.join(self.root_dir, path), self.n_frames, self.frame_step)
        frames = torch.stack(frames)
        return frames, label

In [359]:
import pandas as pd

In [360]:
df = pd.read_csv("../data/mirror-data.csv")
df["Action"] = df["Action"].str.rstrip()
df = df[df.Action != "Talking&Yawning"]
df["label"] = df.Action.astype('category').cat.codes

In [361]:
i = 0
dfTrain = pd.DataFrame()
dfTest = pd.DataFrame()

while i<len(df):
    if i%5==0:
        dfTest = pd.concat([dfTest, df.iloc[[i]]])
    else :
        dfTrain = pd.concat([dfTrain, df.iloc[[i]]])
    i+=1

In [362]:
MAX_SEQ_LENGTH = 8
frame_step = 1

In [363]:
train_ds = VideoDataset(dfTrain, "../data/YawDD/YawDD dataset/Mirror/all/", MAX_SEQ_LENGTH, frame_step)
test_ds = VideoDataset(dfTest, "../data/YawDD/YawDD dataset/Mirror/all/", MAX_SEQ_LENGTH, frame_step)

In [364]:
from torch.utils.data import DataLoader

In [365]:
train_loader = DataLoader(train_ds, batch_size=4)
val_loader = DataLoader(test_ds, batch_size=4)

In [366]:
import torch.optim as optim
from torch import nn
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [367]:
num_epochs = 10

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
    # Evaluate on validation set
    correct = 0
    total = 0
    with torch.no_grad():
        for data in val_loader:
            inputs, labels = data
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_acc = 100 * correct / total
    
    print('[%d] loss: %.3f, val_acc: %.3f' %
          (epoch + 1, running_loss / len(train_loader), val_acc))

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:8                                                                                    │
│                                                                                                  │
│    5 │   for i, data in enumerate(train_loader, 0):                                              │
│    6 │   │   inputs, labels = data                                                               │
│    7 │   │   optimizer.zero_grad()                                                               │
│ ❱  8 │   │   outputs = model(inputs)                                                             │
│    9 │   │   loss = criterion(outputs, labels)                                                   │
│   10 │   │   loss.backward()                                                                     │
│   11 │   │   optimizer.step()                                                                    │
│                                                                                                  │
│ c:\Users\xuton\Documents\Work\Year                                                               │
│ 5\Capstone\Code\se4450-project-group-23\model\.venv\lib\site-packages\torch\nn\modules\module.py │
│ :1501 in _call_impl                                                                              │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1504 │   │   backward_pre_hooks = []                                                           │
│                                                                                                  │
│ c:\Users\xuton\Documents\Work\Year                                                               │
│ 5\Capstone\Code\se4450-project-group-23\model\.venv\lib\site-packages\transformers\models\timesf │
│ ormer\modeling_timesformer.py:767 in forward                                                     │
│                                                                                                  │
│   764 │   │   ```"""                                                                             │
│   765 │   │   return_dict = return_dict if return_dict is not None else self.config.use_return   │
│   766 │   │                                                                                      │
│ ❱ 767 │   │   outputs = self.timesformer(                                                        │
│   768 │   │   │   pixel_values,                                                                  │
│   769 │   │   │   output_attentions=output_attentions,                                           │
│   770 │   │   │   output_hidden_states=output_hidden_states,                                     │
│                                                                                                  │
│ c:\Users\xuton\Documents\Work\Year                                                               │
│ 5\Capstone\Code\se4450-project-group-23\model\.venv\lib\site-packages\torch\nn\modules\module.py │
│ :1501 in _call_impl                                                                              │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backwa

In [163]:
from transformers import TrainingArguments, Trainer

In [164]:
model_name = "TimeSformer"
new_model_name = f"{model_name}-finetuned"
batch_size = 4
num_epochs = 25

args = TrainingArguments(
    new_model_name,
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    max_steps=(train_ds.__len__() // batch_size) * num_epochs,
)

In [165]:
import evaluate

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [166]:
def collate_fn(examples):
    # permute to (num_frames, num_channels, height, width)
    pixel_values = torch.stack(
        [example["video"].permute(1, 0, 2, 3) for example in examples]
    )
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

In [167]:
from transformers import AutoImageProcessor
image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base-finetuned-kinetics")

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [168]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
)

In [169]:
train_results = trainer.train()

c:\Users\xuton\Documents\Work\Year 5\Capstone\Code\se4450-project-group-23\model\.venv\lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/1525 [00:00<?, ?it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 train_results = trainer.train()                                                              │
│   2                                                                                              │
│                                                                                                  │
│ c:\Users\xuton\Documents\Work\Year                                                               │
│ 5\Capstone\Code\se4450-project-group-23\model\.venv\lib\site-packages\transformers\trainer.py:16 │
│ 64 in train                                                                                      │
│                                                                                                  │
│   1661 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1662 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1663 │   │   )                                                                                 │
│ ❱ 1664 │   │   return inner_training_loop(                                                       │
│   1665 │   │   │   args=args,                                                                    │
│   1666 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1667 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ c:\Users\xuton\Documents\Work\Year                                                               │
│ 5\Capstone\Code\se4450-project-group-23\model\.venv\lib\site-packages\transformers\trainer.py:19 │
│ 09 in _inner_training_loop                                                                       │
│                                                                                                  │
│   1906 │   │   │   │   rng_to_sync = True                                                        │
│   1907 │   │   │                                                                                 │
│   1908 │   │   │   step = -1                                                                     │
│ ❱ 1909 │   │   │   for step, inputs in enumerate(epoch_iterator):                                │
│   1910 │   │   │   │   total_batched_samples += 1                                                │
│   1911 │   │   │   │   if rng_to_sync:                                                           │
│   1912 │   │   │   │   │   self._load_rng_state(resume_from_checkpoint)                          │
│                                                                                                  │
│ c:\Users\xuton\Documents\Work\Year                                                               │
│ 5\Capstone\Code\se4450-project-group-23\model\.venv\lib\site-packages\torch\utils\data\dataloade │
│ r.py:634 in __next__                                                                             │
│                                                                                                  │
│    631 │   │   │   if self._sampler_iter is None:                                                │
│    632 │   │   │   │   # TODO(https://github.com/pytorch/pytorch/issues/76750)                   │
│    633 │   │   │   │   self._reset()  # type: ignore[call-arg]                                   │
│ ❱  634 │   │   │   data = self._next_data()                                                      │
│    635 │   │   │   self._num_yielded += 1                                                        │
│    636 │   │   │   if self._dataset_kind == _DatasetKind.It